# Task

-  extract Point of Interest (POI) Names and Street Names from unformatted Indonesia addresses.

# Submisison

- POI/street
- can be empty for POI or street. Both can also be empty which = "/"
- auto complete missing words

# Evaluation

- average of accuracy score of each address

# [AddressNet](https://github.com/jasonrig/address-net)

- Encode each character that includes punctuation with 8 bit vector
- Feed through Bi-directional GRU then pass through dense layer and softmax to predict 2 class which is POI and street name
- rely on data augmentation
- add noise randomly with typo

# [DL Specific Info Extraction](https://towardsdatascience.com/deep-learning-for-specific-information-extraction-from-unstructured-texts-12c5b9dceada)

- Create language model (ULMFit, ELmo) RNN embedding
- Text vectorization with word2vec, GLoVe, tf-idf
- Associate vector of POI and street name with address vector
- Takes in 3 variable length vector each passed through LSTM: phrase, context, phrase + context
- https://www.linkedin.com/pulse/text-data-processing-deep-learning-word-embeddingrnn-lstm-rahul-jain/


# Using DistilBERT with sentence-transformer

# [Deep Learning for Named Entity Recognition - Kfir Bar](https://www.youtube.com/watch?v=TUXbXwu17KE)

- Use IOB annotation. Beginning of named entity, end of named entity and other
- Create embedding of each word. Can take a window as context. Output will be IOB of interested entity i.e. B-POI, E-POI, B-StreetName, E-StreetName, Other. Softmax final
- RNN might be better where we capture output of previous word + embedding of current word
- Bi-directional LSTM better able to capture right and left
- Can be multiple layer LSTM
- Use CRF as final layer
- Add character based encoding as part of input vector

# [State-of-the-art named entity recognition with BERT](https://www.youtube.com/watch?v=aUGCHVbs6oo)

- Char-CNN-BiLSTM-CRF
- Flatten 4 features Tokens + Casing presence + POS + Char CNN

# [Intro to Deep Learning NLP with PyTorch 05 Bi LSTMs and Named Entity Recognition](https://github.com/PythonWorkshop/intro-to-nlp-with-pytorch/tree/master/Named_Entity_Recognition)


# [NLP Tutorial 16 - CV and Resume Parsing with Custom NER Training with SpaCy](https://kgptalkie.com/resume-and-cv-summarization/)

- purely using spacy

# [Building BERT extraction model](https://www.youtube.com/watch?v=MqQ7rqRllIc)

- https://github.com/abhishekkrthakur/bert-entity-extraction

# [Sequence tagging with tensorflow](https://github.com/guillaumegenthial/sequence_tagging)

In [ ]:
!pip install scipy sklearn torchvision tqdm jdc

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import nltk
import matplotlib.pyplot as plt
%matplotlib notebook
plt.rcParams["figure.figsize"] = [20, 10]

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
import torch
torch.__version__

In [ ]:
# Imports for this tutorial
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.optim as optim
import jdc
import json
from collections import defaultdict, OrderedDict
import math
import numpy as np

torch.manual_seed(1)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

In [ ]:
rootPath = "/kaggle/input/scl2021address-extraction"
# Raw Files
trainDs = pd.read_csv(f"{rootPath}/train.csv")
testDs = pd.read_csv(f"{rootPath}/test.csv")

# Create Training Dataset

In [ ]:
def getLabel(words, poi, street):
    labels = []
    for idx, word in enumerate(words):
        if word in poi and (False if idx == 0 else (labels[idx - 1] == 'B-POI' or labels[idx -1] == 'I-POI')):
            labels.append("I-POI")
        elif word in poi:
            labels.append("B-POI")
        elif word in street and (False if idx == 0 else (labels[idx - 1] == 'B-Street' or labels[idx - 1] == 'I-Street')):
            labels.append("I-Street")
        elif word in street:
            labels.append("B-Street")
        if word not in poi and word not in street:
            labels.append("O")
    return labels


def prepare_sequence(seq, to_ix):
    """
    Convert words or tags to intigers and return a Pytorch tensor.
    :param seq: Sequence of words.
    :type seq: list
    :param to_ix: Dictionary mapping words or tags to intigers.
    :return: A Pytorch tensor of indices.
    :rtype: Tensor
    """
    idxs = [to_ix[w] for w in seq]
    return torch.tensor(idxs, dtype=torch.long)

In [ ]:
from nltk.corpus import stopwords
stop = stopwords.words('indonesian')

# Preprocess
# Remove stop words

trainMini = trainDs
trainMini["poi"] = trainMini["POI/street"].str.split("/").str[0].str.split(" ")
trainMini["street"] = trainMini["POI/street"].str.split("/").str[1].str.split(" ")
trainMini['words'] = trainMini["raw_address"].str.split(r"\W+").apply(lambda x: [word for word in x if word not in (stop)])
trainMini['labels'] = trainMini.apply(lambda x: getLabel(x['words'], x['poi'], x['street']), axis=1)
data = [item for sublist in trainMini[['words', 'labels']].apply(lambda x: list(zip(*x)), axis=1).values for item in sublist]
vocab = set([item for sublist in trainMini['words'].values for item in sublist])

In [ ]:
# Training data split
num_train = math.floor(len(data) * 0.8) # 80% to train
training_data, test_data = data[:num_train], data[num_train:]

In [ ]:
import pickle

with open('/kaggle/working/data.pkl', 'wb') as f:
    pickle.dump(data, f)

with open('/kaggle/working/vocab.pkl', 'wb') as f:
    pickle.dump(vocab, f)
    
with open('/kaggle/working/train.pkl', 'wb') as f:
    pickle.dump(training_data, f)
    
with open('/kaggle/working/test.pkl', 'wb') as f:
    pickle.dump(training_data, f)
    
trainMini.to_csv("/kaggle/working/df.csv", index=False)

# Torch Create

In [ ]:
START_TAG = "<START>"
STOP_TAG = "<STOP>"
EMBEDDING_DIM = 5
HIDDEN_DIM = 4
MINIBATCH_SIZE = 2
LEARNING_WEIGHT = 5e-2
WEIGHT_DECAY = 1e-4

In [ ]:
class BiLSTM_CRF(nn.Module):

    def __init__(self, vocab_size, tag_to_ix, embedding_dim, hidden_dim):
        """Initialize network."""
        super(BiLSTM_CRF, self).__init__()
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.vocab_size = vocab_size
        self.tag_to_ix = tag_to_ix
        self.tagset_size = len(tag_to_ix)

        self.word_embeds = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim // 2,
                            num_layers=1, bidirectional=True)

        # Maps the output of the LSTM into tag space.
        self.hidden2tag = nn.Linear(hidden_dim, self.tagset_size)

        # Matrix of transition parameters.  Entry i,j is the score of
        # transitioning *to* i *from* j.
        self.transitions = nn.Parameter(
            torch.randn(self.tagset_size, self.tagset_size))

        # These two statements enforce the constraint that we never transfer
        # to the start tag and we never transfer from the stop tag
        self.transitions.data[tag_to_ix[START_TAG], :] = -10000
        self.transitions.data[:, tag_to_ix[STOP_TAG]] = -10000

        self.hidden = self.init_hidden()

In [ ]:
%%add_to BiLSTM_CRF

def init_hidden(self):
    """Two tensors to hold hidden states, one for each
    LSTM direction with dimensions of (num_layers, 
    minibatch, hidden_dim)"""
    # Minibatch small because small dataset below
    return (torch.randn(2, 1, self.hidden_dim // 2).to(device),
            torch.randn(2, 1, self.hidden_dim // 2).to(device))

In [ ]:
%%add_to BiLSTM_CRF


def _forward_alg(self, feats):
    """Core magic of the Conditional Random Field.  
    
    Input:
        The word embeddeding vectors for a sentence
    
    Since we’re using PyTorch to compute gradients for us, 
    we technically only need the forward part of the forward-backward 
    algorithm """
    # Do the forward algorithm to compute the partition function
    init_alphas = torch.full((1, self.tagset_size), -10000.).to(device)
    # START_TAG ("<START>") has all of the score.
    init_alphas[0][self.tag_to_ix[START_TAG]] = 0.

    forward_var = init_alphas

    # Iterate through the sentence
    for feat in feats:
        alphas_t = []  # The forward tensors at this timestep
        for next_tag in range(self.tagset_size):
            # broadcast the emission score: it is the same regardless of
            # the previous tag
            emit_score = feat[next_tag].view(
                1, -1).expand(1, self.tagset_size)
            # the ith entry of trans_score is the score of transitioning to
            # next_tag from i
            trans_score = self.transitions[next_tag].view(1, -1)
            # The ith entry of next_tag_var is the value for the
            # edge (i -> next_tag) before we do log-sum-exp
            next_tag_var = forward_var + trans_score + emit_score
            # The forward variable for this tag is log-sum-exp of all the
            # scores.
            alphas_t.append(log_sum_exp(next_tag_var).view(1))
        forward_var = torch.cat(alphas_t).view(1, -1)
    terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
    alpha = log_sum_exp(terminal_var)
    return alpha


In [ ]:
%%add_to BiLSTM_CRF

def _get_lstm_features(self, sentence):
    """Compute output vector of BiLSTM - used in 
    the forward pass of network"""
    self.hidden = self.init_hidden()
    embeds = self.word_embeds(sentence).view(len(sentence), 1, -1)
    lstm_out, self.hidden = self.lstm(embeds, self.hidden)
    lstm_out = lstm_out.view(len(sentence), self.hidden_dim)
    # Map LSTM features into tag space
    lstm_feats = self.hidden2tag(lstm_out)
    return lstm_feats

In [ ]:
%%add_to BiLSTM_CRF

def _score_sentence(self, feats, tags):
    """Gives the score of a provided tag sequence"""
    # Gives the score of a provided tag sequence
    score = torch.zeros(1).to(device)
    tags = torch.cat([torch.tensor([self.tag_to_ix[START_TAG]], dtype=torch.long).to(device), 
                      tags])
    for i, feat in enumerate(feats):
        score = score + \
            self.transitions[tags[i + 1], tags[i]] + feat[tags[i + 1]]
    score = score + self.transitions[self.tag_to_ix[STOP_TAG], tags[-1]]
    return score

In [ ]:
%%add_to BiLSTM_CRF

def _viterbi_decode(self, feats):
    """Implements Viterbi algorithm for finding most likely sequence of labels.
    Used in the forward pass of the network.

    We take the maximum over the previous states as opposed to the sum. 
    Input:
        loglikelihoods: torch tensor.
    Output:
        tuple. The first entry is the loglikelihood of this sequence. The second is 
        the most likely sequence of labels. 
    """
    backpointers = []

    # Initialize the viterbi variables in log space
    init_vvars = torch.full((1, self.tagset_size), -10000.).to(device)
    init_vvars[0][self.tag_to_ix[START_TAG]] = 0

    # forward_var at step i holds the viterbi variables for step i-1
    forward_var = init_vvars
    for feat in feats:
        bptrs_t = []  # holds the backpointers for this step
        viterbivars_t = []  # holds the viterbi variables for this step

        for next_tag in range(self.tagset_size):
            # next_tag_var[i] holds the viterbi variable for tag i at the
            # previous step, plus the score of transitioning
            # from tag i to next_tag.
            # We don't include the emission scores here because the max
            # does not depend on them (we add them in below)
            next_tag_var = forward_var + self.transitions[next_tag]
            best_tag_id = argmax(next_tag_var)
            bptrs_t.append(best_tag_id)
            viterbivars_t.append(next_tag_var[0][best_tag_id].view(1))
        # Now add in the emission scores, and assign forward_var to the set
        # of viterbi variables we just computed
        forward_var = (torch.cat(viterbivars_t) + feat).view(1, -1).to(device)
        backpointers.append(bptrs_t)

    # Transition to STOP_TAG
    terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
    best_tag_id = argmax(terminal_var)
    path_score = terminal_var[0][best_tag_id]

    # Follow the back pointers to decode the best path.
    best_path = [best_tag_id]
    for bptrs_t in reversed(backpointers):
        best_tag_id = bptrs_t[best_tag_id]
        best_path.append(best_tag_id)
    # Pop off the start tag (we dont want to return that to the caller)
    start = best_path.pop()
    assert start == self.tag_to_ix[START_TAG]  # Sanity check
    best_path.reverse()
    return path_score, best_path

In [ ]:
%%add_to BiLSTM_CRF

def neg_log_likelihood(self, sentence, tags):
    """Calculate the negative log likelihood given a sequence and labels.
    This is used in training (only) because we don't need to create
    and check the B-I-O tags themselves - only the score is important
    here for calculating the loss."""
    feats = self._get_lstm_features(sentence)
    forward_score = self._forward_alg(feats)
    gold_score = self._score_sentence(feats, tags)
    return forward_score - gold_score

In [ ]:
%%add_to BiLSTM_CRF

def forward(self, sentence):
    """The forward pass function for training the network.
    This is used in inference only."""
    # Get the emission scores (output layer) from the 
    # BiLSTM 
    lstm_feats = self._get_lstm_features(sentence)

    # Find the best path, given the features.
    score, tag_seq = self._viterbi_decode(lstm_feats)
    return score, tag_seq

In [ ]:
# Create a lookup dict for all possible words and record their index
word_to_ix = {k: v for (k, v) in zip(vocab, range(len(vocab)))}
tag_to_ix = {"B": 0, "I": 1, "O": 2, START_TAG: 3, STOP_TAG: 4}
ix_to_tag = {0: "B", 1: "I", 2: "O"}

In [ ]:
model = BiLSTM_CRF(len(word_to_ix), tag_to_ix, EMBEDDING_DIM, HIDDEN_DIM)
model = model.to(device)
optimizer = optim.SGD(model.parameters(), lr=LEARNING_WEIGHT, weight_decay=WEIGHT_DECAY)

# EDA

## Training

In [ ]:
trainDs.head(10)
trainDs.shape

### Word Distribution

In [ ]:
trainAddress = trainDs['raw_address']
words = nltk.tokenize.word_tokenize(trainAddress.str.cat(sep=' '))
word_dist = nltk.FreqDist(words)
rslt = pd.DataFrame(word_dist.most_common(100),
                    columns=['Word', 'Frequency'])
rslt

In [ ]:
rslt.head(40).plot.bar(x="Word", y="Frequency")

### Training POI

In [ ]:
trainPOI = trainDs["POI/street"].str.split("/").str[0].replace('', np.nan).dropna()
top20POI = trainPOI.value_counts()[:20]
top20POI
top20POI.plot(kind="bar")

### Training Street

In [ ]:
trainStreetName = trainDs["POI/street"].str.split("/").str[1].replace('', np.nan).dropna()
top20StreetName = trainStreetName.value_counts()[:20]
top20StreetName
top20StreetName.plot(kind='bar')